In [1]:
from data_loader.dataloader import get_data_loader
from models.vgg_model import VGGModel
from utils.config import process_config
from utils.dirs import create_dirs
from utils.utils import get_args
from keras.models import load_model
import cv2
from keras.preprocessing.image  import ImageDataGenerator, img_to_array
import numpy as np
from sklearn import metrics
from IPython.display import Image
import os, random
from tensorflow.python.client import device_lib

Using TensorFlow backend.
/home/allparel/miniconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
configFile="configs/6.867/matrix_turtleneck.json"
#configFile = "configs/neck_config.json"
config = process_config(configFile)
print(configFile)

model = VGGModel(config)
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(device_lib.list_local_devices())
model.load("/home/allparel/Allparel-ML/experiments/2018-12-11/matrix_turtleneck/checkpoints/matrix_turtleneck-28-0.57.hdf5")

configs/6.867/matrix_turtleneck.json
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56,

In [3]:
generator = get_data_loader(config, False)
batch_size = config.trainer.batch_size
steps = config.trainer.validation_steps
probabilities = model.model.predict_generator(generator, steps, workers=6, use_multiprocessing=True, verbose=1)
val_preds = np.argmax(probabilities, axis=-1)

Found 115371 images belonging to 2 classes.


/home/allparel/miniconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:380: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


50/50 [==============================] - 17s 341ms/step


In [4]:
labels = [""] * len(generator.class_indices)
for label, index in generator.class_indices.items():
    labels[index] = label

In [5]:
val_trues = [0] * batch_size * steps
generator = get_data_loader(config, False)
for i in range(steps):
    x = generator[i][1]
    for j in range(batch_size):
        val_trues[i * batch_size + j] = np.argmax(x[j], axis=-1)

Found 115371 images belonging to 2 classes.


In [6]:
cm = metrics.confusion_matrix(val_trues, val_preds)
print(labels)
print(cm)
print(val_preds)
print(metrics.classification_report(val_trues, val_preds))
#print(probabilities)

['other', 'turtleneck']
[[3179    0]
 [  21    0]]
[0 0 0 ... 0 0 0]
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      3179
          1       0.00      0.00      0.00        21

avg / total       0.99      0.99      0.99      3200



/home/allparel/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
#filename = "sweetheart.jpg"
#filename = "/home/allparel/Allparel-ML/datasets/validation/neck/sweetheartneck/bardot-love-triangle-sweetheart-mini-lace-dress.jpg"
import json
pos_dir = "/home/allparel/6.867Project/true_validation/turtleneck/"
neg_dir = "/home/allparel/6.867Project/true_validation/turtleneck_other/"

pos_files = [pos_dir + f for f in os.listdir(pos_dir) if f.endswith('.jpg')]
neg_files = [neg_dir + f for f in os.listdir(neg_dir) if f.endswith('.jpg')]


#filename = dir + random.choice(os.listdir(dir))
#description_file = "/home/allparel/Allparel-ML/datasets/images/" + os.path.basename(filename).replace(".jpg", ".txt")
    
#with open(description_file, 'r') as f:
    #json_data = json.load(f)
    #print(json_data['description'])
    
#print(filename)
#Image(filename, width=200)



In [9]:
def predict(filename):
    X = np.empty((1, 224, 224, 3))
    image = cv2.imread(filename)
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    X[0,] = np.array(image, dtype="float") / 255.0
    prediction = model.predict(X)
    #print(prediction)
    cls = np.argmax(prediction, axis=-1)[0]
    #print("prediction: ", cls, "confidence", prediction[0][cls], " label: ", labels[cls])
    return (cls, prediction[0][cls])


pos_correct = 0
neg_correct = 0
for filename in pos_files:
    label, conf = predict(filename)
    if label == 1:
        pos_correct += 1
    else:
        print("false negative", filename, conf)
for filename in neg_files:
    label, conf = predict(filename)
    if label == 0:
        neg_correct += 1
        print("true negative", filename, conf)
    else:
        print("false positive", filename, conf)
print("Accuracies", pos_correct/len(pos_files), neg_correct/len(neg_files))
        


false negative /home/allparel/6.867Project/true_validation/turtleneck/ply-cashmere-cashmere-oversized-shirttail-sweater.jpg 0.5658405
false negative /home/allparel/6.867Project/true_validation/turtleneck/etro-short-sleeve-turtleneck-top.jpg 0.56584066
false negative /home/allparel/6.867Project/true_validation/turtleneck/pierre-balmain-chunky-knit-cropped-sweater.jpg 0.56584066
false negative /home/allparel/6.867Project/true_validation/turtleneck/organic-by-john-patrick-turtleneck-wool-mini-dress.jpg 0.5658407
false negative /home/allparel/6.867Project/true_validation/turtleneck/miss-selfridge-exclusive-balloon-sleeve-sweater.jpg 0.5658409
false negative /home/allparel/6.867Project/true_validation/turtleneck/st-john-liquid-satin-turtleneck-blouse.jpg 0.5658405
false negative /home/allparel/6.867Project/true_validation/turtleneck/richard-james-degrade-turtleneck-wool-jumper.jpg 0.56584096
false negative /home/allparel/6.867Project/true_validation/turtleneck/michael-kors-ribbed-stretch-kn

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/jane-bleecker-flannel-romper-3501259.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/lush-side-tie-dress.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/hatley-bluebirds-dress.jpg 0.565841
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/marvy-fashion-yellow-sheer-top.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/brazil-wear-womens-leila-legging.jpg 0.5658411
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/charter-club-lexington-straight-leg-jeans.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/lysse-womens-soho-canvas-crop.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/nina-ricci-pleated-sleeveless-dress.jpg 0.5658409
true negative /home/allparel/6.867Pr

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/kubban-chest-pocket-t-shirt.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/boohoo-corin-frill-skater-tea-dress.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/goen-j-asymmetric-ruffled-silk-skirt.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/chloe-flared-leg-wool-crepe-trousers.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/versace-green-nylon-bomber-jacket.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/stine-goya-virgo-appliqued-silk-shirt-black.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/kingsland-textured-polo-shirt.jpg 0.56584054
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/s-maxmara-velvet-button-up-jacket.jpg 0.5658406
true 

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/palmer-harding-reba-shirt.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/rockport-mens-kingstin-ubal-shoe-us.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/tory-burch-kaylee-silk-shirt-dress.jpg 0.56584024
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/nally-millie-reversible-printed-pencil-skirt.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/tomorrowland-boxy-fit-shirt.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/3-1-phillip-lim-black-high-waisted-wool-trousers.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/lisa-rinna-collection-stretch-denim-slim-legankle-jeans.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/moussy-womens-jasper-s

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/christopher-kane-silk-jacquard-skirt-w-tags.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/amiri-distressed-sweatpants.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/levis-little-boys-4-7x-502-regular-taper-fit-jeans-4.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/asos-tall-stretch-slim-jeans-in-12-5oz-true-black.jpg 0.565841
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/burberry-micro-check-print-cotton-shirt.jpg 0.5658412
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/dr-collection-dr-collection-sleeveless-sheath-dress-plus.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/jason-wu-stretch-wool-skirt.jpg 0.56584007
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/t

...

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/iro-luana-pleated-waist-lace-up-blouse.jpg 0.56584054
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/lace-brief.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/style-co-style-co-ruched-sleeve-top-created-for-macys.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/toscano-mock-neck-long-sleeve-full-zip-jacket.jpg 0.5658407
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/jacob-cohen-patterned-shorts.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/crochet-playsuit.jpg 0.56584096
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/under-armour-golf-coolswitch-fade-graphic-short-sleeve-polo-shirt.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/polo-ralph-lauren-gingham-logoed-shir

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/h-m-slim-ankle-jeans.jpg 0.5658411
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/st-johns-bay-short-sleeve-v-neck-t-shirt-womens-petites.jpg 0.5658407
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/blue-blue-japan-white-drawstring-blouse.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/paisley-gray-mens-slim-fit-flat-front-pindot-suit-separate-pant.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/fabiana-filippi-mid-length-v-neck-cardigan.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/jeremiah-mens-penn-slub-jersey-henley.jpg 0.5658405
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/norrona-trollveggen-thermal-pro-fleece-jacket-mens.jpg 0.5658405
true negative /home/allparel/6.867Project/true_validation/turtle

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/petite-black-wide-leg-jeans.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/no-21-floral-jumper.jpg 0.5658407
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/burberry-little-boys-william-check-collar-polo.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/aidan-mattox-ivory-sequin-halter-top-with-high-low-black-mesh-skirt.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/winona-australia-carmen-mini-dress.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/mih-jeans-boysy-embroidered-linen-blend-dress.jpg 0.56584084
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/hatley-split-neck-shirtdress-womens-dress.jpg 0.5658411
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/b-collection-b

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/cinq-a-sept-lace-trim-cold-shoulder-silk-blouse.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/nike-aeroswift-max-mens-7-running-shorts.jpg 0.5658409
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/good-man-brand-plaid-long-sleeve-slim-fit-shirt.jpg 0.565841
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/saint-laurent-belted-lambskin-jacket.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/halston-satin-shirtdress.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/atm-anthony-thomas-melillo-cotton-trapeze-tank-dress.jpg 0.56584084
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/red-valentino-ruffle-trimmed-cotton-sweater-black.jpg 0.5658405
true negative /home/allparel/6.867Project/true_validation/turtleneck_othe

true negative /home/allparel/6.867Project/true_validation/turtleneck_other/body-glove-mens-grunge-long-sleeve-shirt-8141936.jpg 0.565841
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/rta-cropped-raw-hem-hoodie.jpg 0.5658404
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/giambattista-valli-dress-with-ruffles-and-lace-sleeves.jpg 0.56584066
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/thomas-wylde-asymmetrical-mini-shorts-w-tags.jpg 0.5658408
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/lafayette-148-new-york-desra-long-sleeve-renaissance-paisley-blouse.jpg 0.5658406
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/rebecca-minkoff-gracie-cold-shoulder-sweatshirt.jpg 0.56584084
true negative /home/allparel/6.867Project/true_validation/turtleneck_other/crewcuts-by-j-crew-milton-gingham-check-woven-shirt.jpg 0.5658409
true negative /home/allparel/6.8